<a href="https://colab.research.google.com/github/nguyentrongoanh/CS_weekly_project/blob/Domstests/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

def get_url(url):
    """Get parsed HTML from url
      Input: url to the webpage
      Output: Parsed HTML text of the webpage
    """
    # Send Get request
    r = requests.get(url)

    # Parse HTML text
    soup = BeautifulSoup(r.text, 'html.parser')
    
    return soup

#soup = get_url()

# select the tag that contains the product


def scrape_tiki(url='https://tiki.vn/laptop/c8095?src=c.1846.hamburger_menu_fly_out_banner'):
    """Scrape info of products in Laptop-PC"""
    
    page = 1
    url = url + '&page=' + str(page)
    # Get parsed HTML by calling get_url function
    
    soup = get_url(url)
           
    # Find all products
    products = soup.find_all('div', {'class':'product-item'})
    
    # Store product info in a list
    data = []
    
    page = 1
    
    while products != None and page < 16:
        # Update url including page
        url = url + '&page=' + str(page)
        soup = get_url(url)
        products = soup.find_all('div', {'class':'product-item'})
        # Find all products on current page
        
        # Extract info of each product
        for product in products:    
            d = {'Category':'',
             'Name':'',
             'Final_price':'',
             'Regular_price':'',
             'Discount_percent':'',
             'Installment':'',
             'Cross_border':'',
             'Sponsor':'',
             'Reviews':'',
             'Rating':'',
             'Rating_by_stars':'',
             'Url':'',
             'Image_url':''}

            try:
                d['Category'] = product['data-category']
                #print(d['Category'])
                d['Name'] = product['data-title']
                #print(d['Name'])
                d['Url'] = 'https://tiki.vn' + product.a['href']
                #print(d['Url'])
                d['Image_url'] = product.find('img', {'class':'product-image img-responsive'})['src']
                #print(d['Image'])
                
                try:
                    d['Final_price'] = int(list(product.find('p', {'class':'price-sale'}).text.split())[0].replace('đ', '').replace('.', '', -1))
                    #print(d['Final_Price'])
                    d['Regular_price'] = int(list(product.find('p', {'class':'price-sale'}).text.split())[2].replace('đ', '').replace('.', '', -1))
                    #print(d['Regular_Price'])
                    d['Discount_percent'] = int(list(product.find('p', {'class':'price-sale'}).text.split())[1].replace('-', '').replace('%',''))
                except:
                    d['Regular_price'] = d['Final_price']
                    d['Discount_percent'] = 0
                    
                if product.find('p', {'class':'installment'}):
                    installment = 'YES'
                else:
                    installment = 'NO'
                d['Installment'] = installment
                #print(d['Installment'])

                if 'Quốc Tế' in d['Category']:
                    is_cross_border = 'YES'
                else:
                    is_cross_border = 'NO'
                d['Cross_border'] = is_cross_border
                #print(d['Cross_border'])

                if 'Tài trợ' in product.find('div', {'class':'ship-label-wrapper'}).text:
                    sponsor = 'YES'
                else:
                    sponsor = 'NO'
                #print(sponsor)

                d['Sponsor'] = sponsor

                try:
                    d['Rating'] = int(product.find('span', {'class':'rating-content'}).span['style'][6:-1])
                    d['Rating_by_stars'] = d['Rating'] * 5 / 100
                except:
                    d['Rating'] = 0

                if d['Rating'] == 0:
                    d['Reviews'] = 0
                    d['Rating_by_stars'] = 0
                else:
                    d['Reviews'] = int(product.find('p', {'class':'review'}).text.replace('(', '').split()[0])

                data.append(d)

            except:
                print('Cannot retrieve information')
                #print(page)
                #print(d['Name'])
                #print(d['Discount_percent'])
        
        # Avoid get banned
        sleep(1)
        
        # Increase page
        page += 1

    return data

data = scrape_tiki()
pd.DataFrame(data = data, columns = data[0].keys())

pickledata = pd.DataFrame(data = data, columns = data[0].keys())
# this is a bad name as I am not pickling hehe 

print(len(pickledata))

# print how many items we have in the dataframa



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
pickledata.to_csv("./result3.csv", index=False)
#saving to csv excel file. need to rename the path however